In [1]:
import pandas as pd

In [2]:
train_data = pd.read_csv("C:/Users/susha/Downloads/archive (7)/Genre Classification Dataset/train_data.txt", delimiter=':::', names = ['Sno', 'Name', 'Genre', 'Description'] ,engine='python')
test_data = pd.read_csv("C:/Users/susha/Downloads/archive (7)/Genre Classification Dataset/test_data.txt", delimiter = ':::', names = ['Sno', 'Name', 'Description'], engine='python')
test_data_solution = pd.read_csv("C:/Users/susha/Downloads/archive (7)/Genre Classification Dataset/test_data_solution.txt", delimiter=':::', names = ['Sno', 'Name', 'Genre', 'Description'] ,engine='python')

In [3]:
train_data.head()
test_data.head()
test_data_solution.tail()

,Sno,Name,Genre,Description
54195,54196,"""Tales of Light & Dark"" (2013)",horror,"Covering multiple genres, Tales of Light & Da..."
54196,54197,Der letzte Mohikaner (1965),western,As Alice and Cora Munro attempt to find their...
54197,54198,Oliver Twink (2007),adult,A movie 169 years in the making. Oliver Twist...
54198,54199,Slipstream (1973),drama,"Popular, but mysterious rock D.J Mike Mallard..."
54199,54200,Curitiba Zero Grau (2010),drama,"Curitiba is a city in movement, with rhythms ..."


In [4]:
#code below adds the first 37700 data from test_data_solution into the train datasets and removes those respective data from itself and test_data

last_sno = train_data['Sno'].max()
print(last_sno)

rows_to_append = test_data_solution.head(37700).copy()  # Make a copy to avoid modifying the original DataFrame
rows_to_append.loc[:, 'Sno'] += last_sno + 1  # Use .loc to modify the DataFrame safely

print(rows_to_append)


train_data = train_data.append(rows_to_append)

54214
         Sno                                    Name          Genre  \
0      54216                   Edgar's Lunch (1998)       thriller    
1      54217               La guerra de papá (1977)         comedy    
2      54218            Off the Beaten Track (2010)    documentary    
3      54219                 Meu Amigo Hindu (2015)          drama    
4      54220                      Er nu zhai (1955)          drama    
...      ...                                     ...            ...   
37695  91911                    Fully Loaded (2011)         comedy    
37696  91912                    Tenebrae Lux (2014)         sci-fi    
37697  91913                   Mexican Dance (1898)          short    
37698  91914   Das Lied von den zwei Pferden (2009)    documentary    
37699  91915                  Doin' It Again (2012)    documentary    

                                             Description  
0       L.R. Brane loves his life - his car, his apar...  
1       Spain, March 19

C:\Users\susha\AppData\Local\Temp\ipykernel_15196\1078671139.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(rows_to_append)


In [5]:
test_data = test_data.drop(rows_to_append.index)
test_data_solution = test_data_solution.drop(rows_to_append.index)

In [6]:
test_data_solution.reset_index(drop=True, inplace=True)
test_data_solution.index += 1

In [7]:
test_data.reset_index(drop=True, inplace=True)
test_data.index += 1

In [8]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [9]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\susha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\susha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\susha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\susha\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [10]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\susha\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [11]:
from nltk.stem import WordNetLemmatizer

def preprocess_text(text):
    # Remove special characters, punctuation, and symbols
    text = re.sub(r'[^\w\s]', '', text)

    # Tokenization
    tokens = word_tokenize(text.lower())
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]
    
    # Join tokens back into a string
    preprocessed_text = ' '.join(lemmatized_tokens)
    
    return preprocessed_text

# Apply preprocessing to 'Description' column
train_data['Description'] = train_data['Description'].apply(preprocess_text)
test_data['Description'] = test_data['Description'].apply(preprocess_text)
test_data_solution['Description'] = test_data_solution['Description'].apply(preprocess_text)


In [12]:
X_train = train_data['Description']
y_train = train_data['Genre']
X_test = test_data['Description']
y_test_solution = test_data_solution['Genre']

In [ ]:
from gensim.models import Word2Vec
import numpy as np

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=X_train, vector_size=100, window=5, min_count=1, workers=4)

# Function to calculate document vectors
def calculate_doc_vector(tokens):
    vectors = [word2vec_model.wv[word] for word in tokens if word in word2vec_model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(word2vec_model.vector_size)

# Calculate document vectors for train and test data
X_train_vectors = np.array([calculate_doc_vector(tokens) for tokens in X_train])
X_test_vectors = np.array([calculate_doc_vector(tokens) for tokens in X_test])


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.optimizers import Adam

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform labels
y_train_encoded = label_encoder.fit_transform(y_train)

# Perform the same encoding for test labels if needed
y_test_encoded = label_encoder.transform(y_test_solution)

# Define the model
model = Sequential([
    Dense(512, activation='relu', input_shape=(X_train_vectors.shape[1],)),
    Dropout(0.7),  # Add dropout layer to prevent overfitting
    Dense(256, activation='relu'),
    # Dropout(0.7),
    Dense(128, activation='relu'),
    # Dropout(0.7),

    Dense(64, activation='relu'),
    Dropout(0.7),

    Dense(32, activation='relu'),

    # Dropout(0.7),
    Dense(len(np.unique(y_train)), activation='softmax')  # Assuming y_train is one-hot encoded
])
optimizer = Adam(learning_rate=0.003)

# Compile the model
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()
history = model.fit(X_train_vectors, y_train_encoded, epochs=20, batch_size=32, validation_split=0.1)
